In [4]:
import os
import pickle
import argparse

from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import random

import warnings
warnings.filterwarnings(action='ignore')

import itertools

In [5]:
################################################
# Need to integrate in parser.py for future work
################################################
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

args.master_directory = os.getcwd()
args.basicfilepath = 'data'
args.simulation_directory = 'simulation'
args.variable_save_directory = 'variables'
args.deeplearningmodel_save_directory = 'model'
args.ecl_filename = '2D_ECL'
args.perm_filename = 'PERMX'
args.well_filename = 'WELL'

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120  # ft
args.gridsize_y = 120  # ft

args.time_step = 120  # days
args.total_production_time = 600  # days

args.prod_well_num_max = 5
args.inj_well_num_max = 0
args.total_well_num_max = args.prod_well_num_max + args.inj_well_num_max

args.initial_PRESSURE = 3500  # psi
args.initial_SOIL = 0.75

# For calculation of revenue at each time step
args.oil_price = 60  # $/bbl
args.water_treatment = 3  # $/bbl
args.water_injection = 5  # $/bbl

args.discount_rate = 0.1  # Used for calculation of NPV

In [24]:
well_loc_trial_1 = [(1, 12), (1, 4), (9, 9), (14, 6), (6, 2)]
well_loc_trial_2 = [(8, 8), (5, 1), (1, 2), (14, 6), (2, 12)]
well_loc_trial_3 = [(15, 5), (2, 3), (6, 2), (1, 14), (10, 8)]
well_loc_trial_4 = [(1, 4), (5, 1), (4, 15), (11, 7), (7, 8)]
well_loc_trial_5 = [(1, 14), (1, 3), (14, 1), (10, 8), (6, 2)]

# if not os.path.exists(args.simulation_directory):
#     os.mkdir(args.simulation_directory)
# os.chdir(args.simulation_directory)

well_loc_permut_1 = list(itertools.permutations(well_loc_trial_1))
well_loc_permut_2 = list(itertools.permutations(well_loc_trial_2))
well_loc_permut_3 = list(itertools.permutations(well_loc_trial_3))
well_loc_permut_4 = list(itertools.permutations(well_loc_trial_4))
well_loc_permut_5 = list(itertools.permutations(well_loc_trial_5))

well_loc_permut_all = {}

well_loc_permut_all['well_loc_permut_1'] = well_loc_permut_1
well_loc_permut_all['well_loc_permut_2'] = well_loc_permut_2
well_loc_permut_all['well_loc_permut_3'] = well_loc_permut_3
well_loc_permut_all['well_loc_permut_4'] = well_loc_permut_4
well_loc_permut_all['well_loc_permut_5'] = well_loc_permut_5

for i in range(1, 5+1): # Trial num.
    if not os.path.exists(f'Trial_{i}'):
        os.mkdir(f'Trial_{i}')
    for j in range(len(well_loc_permut_1)):
        output_data_file = []
        output_perm_file = []
        output_well_file = []

        with open(f"{os.path.join(args.basicfilepath, args.ecl_filename)}.TEMPLATE", 'r') as file_read_data:
            line = file_read_data.readline()
            output_data_file.append(line)
            while not line.startswith("[#WELL_DATA]"):
                line = file_read_data.readline()
                output_data_file.append(line)
            line = line.replace("[#WELL_DATA]", f"\'{args.well_filename}_Trial{i}_Comb{j}.DATA\'")
            output_data_file[-1] = line
            while line:
                line = file_read_data.readline()
                output_data_file.append(line)

        with open(f"{os.path.join(args.basicfilepath, args.perm_filename)}.DATA", 'r') as file_read_perm:
            line = file_read_perm.readline()
            output_perm_file.append(line)
            while line:
                line = file_read_perm.readline()
                output_perm_file.append(line)

        sample_data_name = f"{args.ecl_filename}_Trial{i}_Comb{j}.DATA"
        sample_perm_name = f"PERMX.DATA"
        sample_well_name = f"{args.well_filename}_Trial{i}_Comb{j}.DATA"

        with open(f"{os.path.join(f'Trial_{i}', sample_data_name)}", 'w') as file_write_data:
            for l in range(len(output_data_file)):
                file_write_data.write(output_data_file[l])

        with open(f"{os.path.join(f'Trial_{i}', sample_perm_name)}", 'w') as file_write_perm:
            for l in range(len(output_perm_file)):
                file_write_perm.write(output_perm_file[l])

        for k in range(len(well_loc_permut_all[f'well_loc_permut_{i}'][j])):
            output_well_file.append(f"--WELL #{k+1}\n"
                                    f"WELSPECS\n P{k+1} ALL {well_loc_permut_all[f'well_loc_permut_{i}'][j][k][0]} {well_loc_permut_all[f'well_loc_permut_{i}'][j][k][1]} 1* LIQ 3* NO /\n/\n \n"
                                    f"COMPDAT\n P{k+1} {well_loc_permut_all[f'well_loc_permut_{i}'][j][k][0]} {well_loc_permut_all[f'well_loc_permut_{i}'][j][k][1]} 1 1 1* 1* 1* 1 1* 1* 1* Z /\n/\n \n"
                                    f"WCONPROD\n P{k+1} 1* BHP 5000 4* 1500.0 /\n/\n \n"
                                    f"TSTEP\n 1*{args.time_step} /\n \n \n")

        with open(f"{os.path.join(f'Trial_{i}', sample_well_name)}", 'w') as file_write_well:
            for l in range(len(output_well_file)):
                file_write_well.write(output_well_file[l])



In [27]:
for i in range(1, 5+1): # Trial num.
    os.chdir(f'Trial_{i}')
    for j in range(len(well_loc_permut_1)):
        filename = f'2D_ECL_Trial{i}_Comb{j}'
        command = fr"C:\\ecl\\2009.1\\bin\\pc\\eclipse.exe {filename} > NUL"
        os.system(command)
    os.chdir('../')

In [35]:
# Read permeability map
perm_field = []

with open(os.path.join(args.basicfilepath, args.perm_filename + ".DATA"), 'r') as file_read:
    lines = file_read.readlines()
    for line_num in range(args.gridnum_x * args.gridnum_y):
        perm_field.append(float(lines[line_num + 1]))

### Well placement
for trial_num in range(1, 5+1):
    for comb_num in range(1, len(well_loc_permut_1)+1):
        if not os.path.exists(os.path.join('figure', f'Trial{trial_num}-WP')):
            os.mkdir(os.path.join('figure', f'Trial{trial_num}-WP'))
        list_x = []
        list_y = []
        list_num = [1,2,3,4,5]
        for well_location in range(len(well_loc_permut_all[f'well_loc_permut_{trial_num}'][comb_num-1])):
            list_x.append(well_loc_permut_all[f'well_loc_permut_{trial_num}'][comb_num-1][well_location][0]-1) # Grid index: 0~(nx-1)
            list_y.append(well_loc_permut_all[f'well_loc_permut_{trial_num}'][comb_num-1][well_location][1]-1) # Grid index: 0~(ny-1)
        plt.figure(figsize=(8,6)) # Unit: inch
        plt.tight_layout()
        plt.scatter(list_x, list_y, c='k')
        for i, txt in enumerate(list_num):
            # plt.gca().annotate(txt, (list_x[i], list_y[i]))
            plt.gca().text(list_x[i]+0.3, list_y[i]+0.3, txt, fontsize=10)
        plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel("ln(Perm)")
        plt.gca().xaxis.tick_top()
        plt.gca().set_xticks(range(0, 15))
        plt.gca().set_xlabel("Grid X", loc='center')
        plt.gca().xaxis.set_label_position('top')
        plt.gca().set_yticks(range(0, 15))
        plt.gca().set_ylabel("Grid Y")
        plt.gca().set_title(f"ln(Perm) map with well location (Trial {trial_num}, Comb. num. {comb_num-1})", font="Arial", fontsize=16)
        figname = f'Well placement-Trial{trial_num}-Comb{comb_num-1}' + '.png'
        plt.savefig(os.path.join('figure', f'Trial{trial_num}-WP', figname))
        plt.close()


### NPV
production_dict_FOPT = {}
production_dict_FWPT = {}
production_dict_FWIT = {}

for trial_num in range(1, 5+1):
    for comb_num in range(1, len(well_loc_permut_1)+1):
        FOPT_list = []
        FWPT_list = []
        FWIT_list = []
        # Read production data of each sample
        with open(os.path.join(f'Trial_{trial_num}', f"2D_ECL_TRIAL{trial_num}_COMB{comb_num-1}.RSM")) as file_read:
            for i in range(10):
                line = file_read.readline()
            for tstep_idx in range(0, 5+1):
                line_list = [element.strip() for element in line.split()]
                FOPT_list.append(float(line_list[2]))
                FWPT_list.append(float(line_list[3]))
                FWIT_list.append(float(line_list[4]))
                line = file_read.readline()
        production_dict_FOPT[f"TRIAL{trial_num}_COMB{comb_num-1}"] = FOPT_list
        production_dict_FWPT[f"TRIAL{trial_num}_COMB{comb_num-1}"] = FWPT_list
        production_dict_FWIT[f"TRIAL{trial_num}_COMB{comb_num-1}"] = FWIT_list

npv_dict = {}

for trial_num in range(1, 5+1):
    for comb_num in range(1, len(well_loc_permut_1)+1):
        income_list = []
        for tstep_idx in range(0, 5):
            oil_income = (production_dict_FOPT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx+1] - production_dict_FOPT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx]) * args.oil_price / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_treat = (production_dict_FWPT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx+1] - production_dict_FWPT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx]) * args.water_treatment / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_inj = (production_dict_FWIT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx+1] - production_dict_FWIT[f"TRIAL{trial_num}_COMB{comb_num-1}"][tstep_idx]) * args.water_injection / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            income_list.append(oil_income - water_treat - water_inj)
        npv_dict[f"TRIAL{trial_num}_COMB{comb_num-1}"] = list(np.cumsum(np.array(income_list)))

# Plotting NPV of each sample
for trial_num in range(1, 5+1):
    for comb_num in range(1, len(well_loc_permut_1)+1):
        time_step_list = [args.time_step * i for i in range(5+1)]
        plt.figure(figsize=(8,6)) # Unit: inch
        plt.plot(time_step_list, np.array([0] + npv_dict[f"TRIAL{trial_num}_COMB{comb_num-1}"])/(10**6))
        plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
        plt.xticks(time_step_list)
        plt.gca().set_xlabel("Days", loc='center')
        plt.gca().set_ylim(0, 18)
        plt.gca().set_ylabel("NPV, MM$", loc='center')
        plt.grid()
        plt.gca().set_title(f"NPV Value (MM$, Trial {trial_num}, Comb. Num. {comb_num-1})", font="Arial", fontsize=16)
        figname = f'NPV-TRIAL{trial_num}-COMB{comb_num-1}' + '.png'
        plt.savefig(os.path.join('figure', figname))
        plt.close()

# Maximum NPV at each trial
max_npv_dict = {}

for trial_num in range(1, 5+1):
    npv_list = []
    for comb_num in range(1, len(well_loc_permut_1)+1):
        npv_list.append(npv_dict[f"TRIAL{trial_num}_COMB{comb_num-1}"][-1])
    max_npv_idx = npv_list.index(max(npv_list))
    max_npv_value = max(npv_list)
    max_npv_dict[f'Trial{trial_num}_Comb{max_npv_idx}'] = max_npv_value

print(max_npv_dict)

{'Trial1_Comb76': 16023383.001354195, 'Trial2_Comb117': 16048798.659971658, 'Trial3_Comb119': 16093577.670073992, 'Trial4_Comb62': 16009434.744919598, 'Trial5_Comb76': 16077681.994192319}
